In [ ]:
import glob
import re
import pandas as pd
from IPython.core.display import display
from pathlib import Path

data_path = "../allennlp-model/data/train/train.english.v4_gold_conll" # conll-formatted file to read into dataframe
pkl_dest = 'data/dataframes/coref_ontonotes_train.pkl' # pickled dataframe destination

In [ ]:
coref_df = pd.DataFrame(columns=['Doc_ID', 'Text_ID', 'Text', 'Sentence_lengths', 'Coref_Clusters'])
print(coref_df)

In [ ]:
def get_coref_clusters(path, document_id):
    temp_df = pd.DataFrame(columns=['Doc_ID', 'Text_ID', 'Text', 'Sentence_lengths', 'Coref_Clusters'])

    with open(path, 'r') as file:
        lines = file.readlines()

        sentence = []
        text = []
        sentence_lengths = []
        coref_clusters = {}
        temp_clusters = {}
        columns = []

        for line in lines:
            # if we enter a new document
            if line.startswith('#'):
                if text:
                    document_id = columns[0]
                    text_id = columns[1]
                    temp_df = temp_df.append({'Doc_ID': document_id,
                                              'Text_ID': document_id + '-' + str(text_id),
                                              'Text': text,
                                              'Sentence_lengths': sentence_lengths,
                                              'Coref_Clusters': list(coref_clusters.values())},
                                             ignore_index=True)
                    text = []
                    sentence_lengths = []
                    coref_clusters = {}
                    temp_clusters = {}
            elif line == '\n':
                text.append(sentence)
                sentence_lengths.append(int(columns[2]) + 1) #zero indexed in dataset
                sentence = []
            else:
                columns = line.split()
                document_id = columns[0]
                index = int(columns[2]) + sum(sentence_lengths)
                sentence.append(columns[3])
                coref_column = columns[-1]
                if not coref_column == '-':
                    corefs = coref_column.split('|')
                    for coref in corefs:
                        key = coref.strip('()')
                        if re.match(r'\([0-9]+\)', coref):
                            coref_clusters.setdefault(key, []).append([index, index])
                        if re.match(r'^\([0-9]+$', coref):
                            temp_clusters.setdefault(key, []).append(index)
                        if re.match(r'^[0-9]+\)$', coref):
                            temp_clusters.setdefault(key, []).append(index)
                            cluster_span = temp_clusters.pop(key)
                            # om det finns fler än två index i cluster_span
                            # spara det andra i temp cluster
                            if len(cluster_span) > 2:
                                temp_clusters.setdefault(key, []).append(cluster_span.pop(1))
                            coref_clusters.setdefault(key, []).append(cluster_span)
    display(temp_df)
    return temp_df

In [ ]:

#for path in Path(data_path).rglob('*.gold_conll'):
# with open(data_path, 'r') as reader:
#     print(path)
#     filename = str(path).split('/')[-1]
    #document_id = filename.split('.')[0]
df = get_coref_clusters(data_path, 'rand')
coref_df = coref_df.append(df)

In [ ]:
#display(coref_df)
save_ids = ('wsj_2358', 'wsj_2346', 'wsj_2356', 'wsj_2321', 'wsj_2316', 'wsj_2350', 'wsj_2363', 'wsj_2309', 'wsj_2354', 'wsj_2344', 'wsj_2383', 'c2e_0029', 'c2e_0039', 'p2.5_a2e_0039', 'p2.5_c2e_0049', 'c2e/00/c2e_0009', '42/nt_4209', '40/nt_4019', '10/ot_1019', '45/nt_4509', '12/ot_1209', '11/ot_1119', '41/nt_4109', '58/nt_5809', 'sinorama/10/ectb_1069', 'xinhua/01/chtb_0129', '/xinhua/01/chtb_0109')
#print(coref_df.loc[df['Doc_ID'].contains('00/c2e_0009')].loc[331, 'Text'])
df2 = coref_df.loc[coref_df['Doc_ID'].str.endswith(save_ids)]
#df2 = coref_df.loc[any(ext in url_string for ext in extensionsToCheck)]
display(df2)
coref_df.to_pickle(pkl_dest)
df2.to_pickle('../data/dataframes/eng_suc_sim')